In [1]:
%pip install mediapipe opencv-python numpy pandas
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture('smash_video_example.mp4')

INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/50.9 MB ? eta -:--:--
   - -------------------------------------- 2.1/50.9 MB 14.7 MB/s eta 0:00:04
   ---- ----------------------------------- 5.8/50.9 MB 16.8 MB/s eta 0:00:03
   -------- ------------------------------- 10.5/50.9 MB 18.7 MB/s eta 0:00:03
   ----------- ---------------------------- 14.2/50.9 MB 18.2 MB/s eta 0:00:03
   -------------- ------------------------- 18.4/50.9 MB 18.4 MB/s eta 0:00:02
   ----------------- ---------------------- 22.8/50.9 MB 19.0 MB/s eta 0:00:02
   --------------------- ------------------ 27.3/50.9 MB 19.2 MB/s eta 0:00:02
   ------------------------ --------------- 31.5/50.9 MB 19.4 MB/s eta 

In [61]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture('smash_video_example.mp4')

keypoints_data = []
frame_count = 0

# Indices for right shoulder, left shoulder, left elbow, right elbow, left wrist, right wrist, left hip, right hip
KEYPOINT_INDICES = [12, 11, 14, 13, 16, 15, 24, 23]

while cap.isOpened():
    ret, frame = cap.read() 
    if not ret:
        break
    
    # Convert the frame to RGB (MediaPipe requires RGB)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    
    if results.pose_landmarks:
        # Extract only the specified keypoints
        landmarks = results.pose_landmarks.landmark
        keypoints = []
        for idx in KEYPOINT_INDICES:
            lm = landmarks[idx]
            keypoints.extend([lm.x, lm.y])  # store x, y only
        
        keypoints_data.append([frame_count] + keypoints)
        
        # Optional: draw skeleton on video
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    cv2.imshow('Pose Detection', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
        break
    
    frame_count += 1

cap.release()
cv2.destroyAllWindows()

In [62]:
# Build columns for selected keypoints
KEYPOINT_INDICES = [12, 11, 14, 13, 16, 15, 24, 23]
columns = ['frame_count'] + [f'kpt_{i}_{axis}' for i in KEYPOINT_INDICES for axis in ['x', 'y']]

df_user = pd.DataFrame(keypoints_data, columns=columns)
df_user.insert(0, 'id', 'user_video')
df_user.insert(1, 'type_of_shot', 'smash')
print(df_user.head())

           id type_of_shot  frame_count  kpt_12_x  kpt_12_y  kpt_11_x  \
0  user_video        smash           21  0.522793  0.522813  0.526127   
1  user_video        smash           22  0.522095  0.522841  0.527906   
2  user_video        smash           23  0.522939  0.518441  0.527462   
3  user_video        smash           24  0.524083  0.524515  0.526714   
4  user_video        smash           25  0.521340  0.524364  0.527568   

   kpt_11_y  kpt_14_x  kpt_14_y  kpt_13_x  kpt_13_y  kpt_16_x  kpt_16_y  \
0  0.515952  0.526208  0.554479  0.526527  0.548947  0.537674  0.579922   
1  0.515345  0.524518  0.554635  0.527557  0.548424  0.537645  0.580251   
2  0.511081  0.522884  0.554521  0.525389  0.539666  0.537318  0.580299   
3  0.504069  0.519677  0.555680  0.513495  0.515500  0.537137  0.580209   
4  0.510944  0.518640  0.554135  0.524223  0.539779  0.536368  0.579573   

   kpt_15_x  kpt_15_y  kpt_24_x  kpt_24_y  kpt_23_x  kpt_23_y  
0  0.534633  0.573548  0.512356  0.543033  0.5

In [63]:
# Save the filtered keypoints DataFrame to CSV
csv_filename = 'user_keypoints_selected.csv'
df_user.to_csv(csv_filename, index=False)
print(f"Saved keypoints to {csv_filename}")

Saved keypoints to user_keypoints_selected.csv


In [66]:
df = pd.read_csv('user_keypoints_selected.csv')
# Removing type_of_shot (they're all the same)
df = df.drop('type_of_shot', axis=1)
df.head()

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Choose which shot id to visualize
shot_id = 'user_video'
shot_df = df[df['id'] == shot_id].reset_index(drop=True)

# Define which keypoints you actually have in your data
KEYPOINT_INDICES = [12, 11, 14, 13, 16, 15, 24, 23]  # Based on your CSV columns

# Create a mapping from keypoint index to position in your data array
kpt_to_idx = {kpt: i for i, kpt in enumerate(KEYPOINT_INDICES)}

# Skeleton connections (using the actual keypoint numbers)
skeleton = [
    (11, 12),   # left shoulder to right shoulder
    (11, 13),   # left shoulder to left elbow
    (13, 15),   # left elbow to left wrist
    (12, 14),   # right shoulder to right elbow
    (14, 16),   # right elbow to right wrist
    (11, 23),   # left shoulder to left hip
    (12, 24),   # right shoulder to right hip
    (23, 24)    # left hip to right hip
]

# Set up the plot
fig, ax = plt.subplots(figsize=(6, 8))
lines = []
for _ in skeleton:
    line, = ax.plot([], [], 'ro-', markersize=5, linewidth=2)
    lines.append(line)

# Set plot limits based on your data (0-1 normalized coordinates)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.invert_yaxis()
ax.set_title(f"Skeleton Animation - Shot {shot_id}")
ax.set_aspect('equal')

# Animation function
def update(frame):
    row = shot_df.iloc[frame]
    # Build keypoints dictionary
    keypoints = {}
    for kpt in KEYPOINT_INDICES:
        keypoints[kpt] = (row[f'kpt_{kpt}_x'], row[f'kpt_{kpt}_y'])
    
    # Draw skeleton connections
    for idx, (a, b) in enumerate(skeleton):
        if a in keypoints and b in keypoints:
            pt_a = keypoints[a]
            pt_b = keypoints[b]
            if pt_a[0] != 0 and pt_a[1] != 0 and pt_b[0] != 0 and pt_b[1] != 0:
                lines[idx].set_data([pt_a[0], pt_b[0]], [pt_a[1], pt_b[1]])
            else:
                lines[idx].set_data([], [])
        else:
            lines[idx].set_data([], [])
    
    return lines

# Create animation
anim = FuncAnimation(fig, update, frames=len(shot_df), interval=100, blit=True)

# Save animation 
anim.save(f'badminton_shot_{shot_id}.gif', writer='pillow')
plt.close()